## 上部分

In [47]:
import pandas as pd         
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from general_pic_setup import setup_mpl_single2
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.collections import LineCollection

# 样式设置
setup_mpl_single2()
mpl.rcParams['axes.titlesize'] = 'medium'

# 路径配置
script_dir = Path(os.getcwd())
data_dir = script_dir.parent / "data"
output_dir = data_dir / "2-2-Country_Policy_Trends"
output_dir.mkdir(parents=True, exist_ok=True)

# 数据加载与预处理
df = pd.read_parquet(data_dir / "2-1-country_proportions.parquet")
country_col = 'REF_AREA'
year_col = 'TIME_PERIOD'

df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
df = df.dropna(subset=[year_col])
df = df[(df[year_col] >= 2005) & (df[year_col] <= 2023)].sort_values(year_col)

# 按大洲分组的国家代码
continent_countries = {
    "Asia": ["CHN", "JPN", "KOR", "IND", "SAU"],
    "Europe": ["DEU", "FRA", "GBR", "ITA", "ESP", "FIN", "GRC", "ISL"],
    "North America": ["CAN", "MEX", "CRI"],
    "South America": ["ARG", "CHL", "COL", "PER"],
    "Oceania": ["AUS", "NZL"],
    "Africa": ["ZAF"]
}

# 选定的7个国家（按你的要求）
selected_countries = [
    "DEU",  # 德国 - 欧洲
    "JPN",  # 日本 - 亚洲
    "MEX",  # 墨西哥 - 北美洲
    "NZL",  # 新西兰 - 大洋洲
    "CHL",  # 智利 - 南美洲
    "ARG"   # 阿根廷 - 南美洲
]

# 政策配置（已更新，添加Y轴范围）
policies = {
    "Transport – market-based instruments": {
        "label": "Transport Policy",
        "y_range": (0, 0.2)
    },
    "Public Research, Development and Demonstration": {
        "label": "Research Development",
        "y_range": (0, 0.6)
    }
}

# 政策名称到短名称的映射（用于文件命名）
policy_short_mapping = {
    "Transport – market-based instruments": "Transport_MBI",
    "Public Research, Development and Demonstration": "Public_RD"
}

# 绘图参数
BORDER_WIDTH = 0.6
CURVE_WIDTH = 1.5  # 曲线线宽调细
box_height = 0.4
gap_between = 0.01
total_height_per_ridge = box_height + gap_between


def plot_ridge_chart(policy_name, policy_config, selected_countries, df):
    fig, ax = plt.subplots(figsize=(4, 9))
    cmap = cm.get_cmap('coolwarm')
    
    policy_label = policy_config["label"]
    y_min, y_max = policy_config["y_range"]
    
    # 筛选有效国家
    valid_countries = [
        c for c in selected_countries 
        if c in df[country_col].values and 
        not df[df[country_col] == c][policy_name].isna().all()
    ]
    
    total_plot_height = (len(valid_countries) - 1) * total_height_per_ridge + box_height

    for idx, country_code in enumerate(valid_countries):
        df_country = df[df[country_col] == country_code]
        years = df_country[year_col].values
        values = df_country[policy_name].values
        
        # 移除缺失值
        mask = ~np.isnan(values)
        years, values = years[mask], values[mask]
        
        baseline = idx * total_height_per_ridge
        values_clipped = np.clip(values, y_min, y_max)

        # 绘制L型边框（左边+底边）
        ax.plot([2005, 2005], [baseline, baseline + box_height],
                color='black', linewidth=BORDER_WIDTH, zorder=idx - 1)
        ax.plot([2005, 2023], [baseline, baseline],
                color='black', linewidth=BORDER_WIDTH, zorder=idx - 1)

        # 绘制渐变曲线（按照y_range进行归一化映射）
        values_normalized = (values_clipped - y_min) / (y_max - y_min) * box_height
        points = np.array([years, baseline + values_normalized]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        n_segments = len(segments)
        colors_gradient = [cmap(i / (n_segments - 1)) for i in range(n_segments)]
        lc = LineCollection(segments, colors=colors_gradient, 
                          linewidth=CURVE_WIDTH, alpha=0.95, zorder=idx + 100)
        ax.add_collection(lc)

        # 国家标签（左侧）
        norm_y = (baseline + 0.2) / total_plot_height
        ax.text(-0.12, norm_y, f'{country_code}',
                transform=ax.transAxes, va='center', ha='right',
                fontsize=11, color='black', weight='bold')

        # Y轴刻度标签（左侧外部）- 使用实际的y_range值
        ax.text(2004.3, baseline + 0.02, f'{y_min}',
                va='center', ha='right', fontsize=8, color='black')
        ax.text(2004.3, baseline + 0.38, f'{y_max}',
                va='center', ha='right', fontsize=8, color='black')

    # X轴设置
    ax.set_xlim(2005, 2023)
    ax.set_xticks([2005, 2014, 2023])
    ax.set_xticklabels([2005, 2014, 2023], fontsize=11, rotation=60, ha='center')
    ax.set_xlabel('Year', fontsize=13, labelpad=3)

    # Y轴设置
    total_y_max = (len(valid_countries) - 1) * total_height_per_ridge + box_height
    ax.set_ylim(0, total_y_max)
    ax.set_yticks([])
    ax.set_ylabel('Countries', fontsize=13, labelpad=55)

    # 边框设置（只保留底部）
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(BORDER_WIDTH)
    ax.tick_params(axis='x', colors='black', width=BORDER_WIDTH, length=4, pad=2)

    plt.tight_layout()

    # 保存图片（使用新的政策短名称映射）
    policy_short = policy_short_mapping[policy_name]
    out_path = output_dir / f"{policy_short}_ridge_plot.png"
    fig.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.15)
    plt.close()
    print(f"  ✓ {out_path.name} 已保存 ({len(valid_countries)} 个国家)")


# 主程序
print(f"\n开始生成山脊图...")
print(f"选定国家: {', '.join(selected_countries)}")
print(f"  - DEU (德国, 欧洲)")
print(f"  - AUS (澳大利亚, 大洋洲)")
print(f"  - JPN (日本, 亚洲)")
print(f"  - CHL (智利, 南美洲)")
print(f"  - MEX (墨西哥, 北美洲)")
print(f"  - NZL (新西兰, 大洋洲)")
print(f"  - ARG (阿根廷, 南美洲)\n")

for policy_idx, (policy_name, policy_config) in enumerate(policies.items(), 1):
    print(f"[{policy_idx}/{len(policies)}] {policy_name}")
    plot_ridge_chart(policy_name, policy_config, selected_countries, df)

print(f"\n✅ 完成！共生成 {len(policies)} 个山脊图")
print(f"📁 保存位置: {output_dir}\n")



开始生成山脊图...
选定国家: DEU, JPN, MEX, NZL, CHL, ARG
  - DEU (德国, 欧洲)
  - AUS (澳大利亚, 大洋洲)
  - JPN (日本, 亚洲)
  - CHL (智利, 南美洲)
  - MEX (墨西哥, 北美洲)
  - NZL (新西兰, 大洋洲)
  - ARG (阿根廷, 南美洲)

[1/2] Transport – market-based instruments
  ✓ Transport_MBI_ridge_plot.png 已保存 (6 个国家)
[2/2] Public Research, Development and Demonstration


C:\Users\28274\AppData\Local\Temp\ipykernel_17968\3361538708.py:78: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')
C:\Users\28274\AppData\Local\Temp\ipykernel_17968\3361538708.py:78: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


  ✓ Public_RD_ridge_plot.png 已保存 (6 个国家)

✅ 完成！共生成 2 个山脊图
📁 保存位置: f:\Desktop\CAMPF_Supplementary\data\2-2-Country_Policy_Trends



## 全体国家都画

In [ ]:
import pandas as pd          
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from general_pic_setup import setup_mpl_single2
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.collections import LineCollection

# 样式设置
setup_mpl_single2()
mpl.rcParams['axes.titlesize'] = 'medium'

# 路径配置
script_dir = Path(os.getcwd())
data_dir = script_dir.parent / "data"
output_dir = data_dir / "2-2-Country_Policy_Trends"
output_dir.mkdir(parents=True, exist_ok=True)

# 数据加载与预处理
df = pd.read_parquet(data_dir / "2-1-country_proportions.parquet")
country_col = 'REF_AREA'
year_col = 'TIME_PERIOD'

df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
df = df.dropna(subset=[year_col])
df = df[(df[year_col] >= 2005) & (df[year_col] <= 2023)].sort_values(year_col)

# 选择49个国家
selected_countries = [
    "ARG", "AUS", "AUT", "BEL", "BGR", "CAN", "CHE", "CHL", "CHN", "COL",
    "CRI", "CZE", "DEU", "DNK", "ESP", "EST", "FIN", "FRA", "GBR", "GRC",
    "HRV", "HUN", "IDN", "IND", "IRL", "ISL", "ISR", "ITA", "JPN", "KOR",
    "LTU", "LUX", "LVA", "MEX", "MLT", "NLD", "NOR", "NZL", "PER", "POL",
    "PRT", "ROU", "RUS", "SAU", "SVK", "SVN", "SWE", "TUR", "ZAF"
]

# 政策配置（已更新，添加Y轴范围）
policies = {
    "Transport – market-based instruments": {
        "label": "Transport Policy",
        "y_range": (0, 0.2)
    },
    "Public Research, Development and Demonstration": {
        "label": "Research Development",
        "y_range": (0, 0.6)
    }
}

# 政策名称到短名称的映射（用于文件命名）
policy_short_mapping = {
    "Transport – market-based instruments": "Transport_MBI",
    "Public Research, Development and Demonstration": "Public_RD"
}

# 绘图参数
BORDER_WIDTH = 0.6
CURVE_WIDTH = 1.5  # 曲线线宽调细
box_height = 0.4
gap_between = 0.01
total_height_per_ridge = box_height + gap_between


def plot_ridge_chart(policy_name, policy_config, selected_countries, df):
    cmap = cm.get_cmap('coolwarm')
    
    policy_label = policy_config["label"]
    y_min, y_max = policy_config["y_range"]
    
    # 筛选有效国家
    valid_countries = [
        c for c in selected_countries 
        if c in df[country_col].values and 
        not df[df[country_col] == c][policy_name].isna().all()
    ]
    
    # 动态计算图的高度
    num_countries = len(valid_countries)
    total_plot_height = (num_countries - 1) * total_height_per_ridge + box_height
    
    # 根据国家数量动态调整figsize高度
    # 基础高度 + 每个国家增加的高度
    fig_height = max(6, num_countries * 0.25)  # 每个国家约0.25英寸，最小6英寸
    fig, ax = plt.subplots(figsize=(4, fig_height))

    for idx, country_code in enumerate(valid_countries):
        df_country = df[df[country_col] == country_code]
        years = df_country[year_col].values
        values = df_country[policy_name].values
        
        # 移除缺失值
        mask = ~np.isnan(values)
        years, values = years[mask], values[mask]
        
        baseline = idx * total_height_per_ridge
        values_clipped = np.clip(values, y_min, y_max)

        # 绘制L型边框（左边+底边）
        ax.plot([2005, 2005], [baseline, baseline + box_height],
                color='black', linewidth=BORDER_WIDTH, zorder=idx - 1)
        ax.plot([2005, 2023], [baseline, baseline],
                color='black', linewidth=BORDER_WIDTH, zorder=idx - 1)

        # 绘制渐变曲线（按照y_range进行归一化映射）
        values_normalized = (values_clipped - y_min) / (y_max - y_min) * box_height
        points = np.array([years, baseline + values_normalized]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        n_segments = len(segments)
        colors_gradient = [cmap(i / (n_segments - 1)) for i in range(n_segments)]
        lc = LineCollection(segments, colors=colors_gradient, 
                          linewidth=CURVE_WIDTH, alpha=0.95, zorder=idx + 100)
        ax.add_collection(lc)

        # 国家标签（左侧）
        norm_y = (baseline + 0.2) / total_plot_height
        ax.text(-0.12, norm_y, f'{country_code}',
                transform=ax.transAxes, va='center', ha='right',
                fontsize=9, color='black', weight='bold')  # 字号略小以适应更多国家

        # Y轴刻度标签（左侧外部）- 使用实际的y_range值
        ax.text(2004.3, baseline + 0.02, f'{y_min}',
                va='center', ha='right', fontsize=7, color='black')
        ax.text(2004.3, baseline + 0.38, f'{y_max}',
                va='center', ha='right', fontsize=7, color='black')

    # X轴设置
    ax.set_xlim(2005, 2023)
    ax.set_xticks([2005, 2014, 2023])
    ax.set_xticklabels([2005, 2014, 2023], fontsize=11, rotation=60, ha='center')
    ax.set_xlabel('Year', fontsize=13, labelpad=3)

    # Y轴设置
    total_y_max = (len(valid_countries) - 1) * total_height_per_ridge + box_height
    ax.set_ylim(0, total_y_max)
    ax.set_yticks([])
    ax.set_ylabel('Countries', fontsize=13, labelpad=55)

    # 边框设置（只保留底部）
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(BORDER_WIDTH)
    ax.tick_params(axis='x', colors='black', width=BORDER_WIDTH, length=4, pad=2)

    plt.tight_layout()

    # 保存图片（使用新的政策短名称映射）
    policy_short = policy_short_mapping[policy_name]
    out_path = output_dir / f"{policy_short}_ridge_plot.png"
    fig.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.15)
    plt.close()
    print(f"  ✓ {out_path.name} 已保存 ({len(valid_countries)} 个国家，图高: {fig_height:.1f}英寸)")


# 主程序
print(f"\n开始生成山脊图...")
print(f"选定国家数量: {len(selected_countries)} 个\n")

for policy_idx, (policy_name, policy_config) in enumerate(policies.items(), 1):
    print(f"[{policy_idx}/{len(policies)}] {policy_name}")
    plot_ridge_chart(policy_name, policy_config, selected_countries, df)

print(f"\n✅ 完成！共生成 {len(policies)} 个山脊图")
print(f"📁 保存位置: {output_dir}\n")



开始生成山脊图...
选定国家数量: 49 个

[1/2] Transport – market-based instruments


C:\Users\28274\AppData\Local\Temp\ipykernel_17968\1414760580.py:66: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


  ✓ Transport_MBI_ridge_plot.png 已保存 (49 个国家，图高: 12.2英寸)
[2/2] Public Research, Development and Demonstration


C:\Users\28274\AppData\Local\Temp\ipykernel_17968\1414760580.py:66: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


  ✓ Public_RD_ridge_plot.png 已保存 (49 个国家，图高: 12.2英寸)

✅ 完成！共生成 2 个山脊图
📁 保存位置: f:\Desktop\CAMPF_Supplementary\data\2-2-Country_Policy_Trends



## 最终版

In [59]:
import pandas as pd          
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from general_pic_setup import setup_mpl_single2
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.collections import LineCollection

# 样式设置
setup_mpl_single2()
mpl.rcParams['axes.titlesize'] = 'medium'

# 路径配置
script_dir = Path(os.getcwd())
data_dir = script_dir.parent / "data"
output_dir = data_dir / "2-2-Country_Policy_Trends"
output_dir.mkdir(parents=True, exist_ok=True)

# 数据加载与预处理
df = pd.read_parquet(data_dir / "2-1-country_proportions.parquet")
country_col = 'REF_AREA'
year_col = 'TIME_PERIOD'

df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
df = df.dropna(subset=[year_col])
df = df[(df[year_col] >= 2005) & (df[year_col] <= 2023)].sort_values(year_col)

# （保留但当前未使用）按大洲分组
continent_countries = {
    "Asia": ["CHN", "JPN", "KOR", "IND", "SAU"],
    "Europe": ["DEU", "FRA", "GBR", "ITA", "ESP", "FIN", "GRC", "ISL"],
    "North America": ["CAN", "MEX", "CRI"],
    "South America": ["ARG", "CHL", "COL", "PER"],
    "Oceania": ["AUS", "NZL"],
    "Africa": ["ZAF"]
}

# 选定的 6 个国家（从下往上依次是：Cluster, DEU, JPN, MEX, NZL, CHL, ARG）
selected_countries = [
    "DEU",  # 德国 - 欧洲
    "JPN",  # 日本 - 亚洲
    "MEX",  # 墨西哥 - 北美洲
    "NZL",  # 新西兰 - 大洋洲
    "CHL",  # 智利 - 南美洲
    "ARG"   # 阿根廷 - 南美洲
]

# 政策配置（已更新，添加Y轴范围）
policies = {
    "Transport – market-based instruments": {
        "label": "Transport Policy",
        "y_range": (0, 0.2)
    },
    "Public Research, Development and Demonstration": {
        "label": "Research Development",
        "y_range": (0, 0.6)
    }
}

# 政策名称到短名称的映射（用于文件命名）
policy_short_mapping = {
    "Transport – market-based instruments": "Transport_MBI",
    "Public Research, Development and Demonstration": "Public_RD"
}

# 绘图参数
BORDER_WIDTH = 0.6
CURVE_WIDTH = 1.5   # 曲线线宽
box_height = 0.4    # 普通国家框高度
gap_between = 0.01  # 框之间间隔
empty_box_extra_height = 0.3  # Cluster 额外高度 (+0.3)


def plot_ridge_chart(policy_name, policy_config, selected_countries, df):
    fig, ax = plt.subplots(figsize=(4, 9))
    cmap = cm.get_cmap('coolwarm')
    
    y_min, y_max = policy_config["y_range"]
    
    # 先筛掉完全没有数据的国家（保持你原来的逻辑）
    valid_countries = [
        c for c in selected_countries
        if c in df[country_col].values
        and not df[df[country_col] == c][policy_name].isna().all()
    ]
    
    # 图中的层顺序：最下面是 Cluster，其上依次是 valid_countries（DEU, JPN, ...）
    layers = ["CLUSTER"] + valid_countries
    
    # 为每一层设置高度：Cluster 高 0.4 + 0.3，其余 0.4
    heights = []
    for code in layers:
        if code == "CLUSTER":
            heights.append(box_height + empty_box_extra_height)
        else:
            heights.append(box_height)
    
    # 计算每一层的 baseline，使得相邻框之间的间隔恒为 gap_between
    baselines = [0.0] * len(layers)
    for i in range(1, len(layers)):
        baselines[i] = baselines[i-1] + heights[i-1] + gap_between
    
    # 整个图的总高度（用于 y 轴范围 & 标签归一化）
    if layers:
        total_plot_height = baselines[-1] + heights[-1]
    else:
        total_plot_height = 1.0  # 兜底，基本不会走到这里
    
    for idx, country_code in enumerate(layers):
        baseline = baselines[idx]
        current_box_height = heights[idx]
        is_cluster = (country_code == "CLUSTER")
        
        # 绘制 L 型边框（左边 + 底边）
        ax.plot([2005, 2005], [baseline, baseline + current_box_height],
                color='black', linewidth=BORDER_WIDTH, zorder=idx - 1)
        ax.plot([2005, 2023], [baseline, baseline],
                color='black', linewidth=BORDER_WIDTH, zorder=idx - 1)

        # 只有非 Cluster 才绘制曲线
        if not is_cluster:
            df_country = df[df[country_col] == country_code]
            years = df_country[year_col].values
            values = df_country[policy_name].values

            # 移除缺失值
            mask = ~np.isnan(values)
            years, values = years[mask], values[mask]

            if len(values) > 1:
                values_clipped = np.clip(values, y_min, y_max)
                values_normalized = (values_clipped - y_min) / (y_max - y_min) * box_height

                points = np.array([years, baseline + values_normalized]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                n_segments = len(segments)
                if n_segments > 1:
                    colors_gradient = [cmap(i / (n_segments - 1)) for i in range(n_segments)]
                else:
                    colors_gradient = [cmap(0.5)]
                lc = LineCollection(segments, colors=colors_gradient,
                                    linewidth=CURVE_WIDTH, alpha=0.95, zorder=idx + 100)
                ax.add_collection(lc)

        # 国家 / Cluster 标签（左侧），标签高度保持“正常”，不居中
        label_y_offset = 0.2
        norm_y = (baseline + label_y_offset) / total_plot_height
        display_label = "Cluster" if is_cluster else country_code
        ax.text(-0.12, norm_y, display_label,
                transform=ax.transAxes, va='center', ha='right',
                fontsize=11, color='black', weight='bold')

        # Y 轴刻度标签（左侧外部）
        # Cluster 的顶部刻度在 current_box_height 顶部，其余保持原来 0.38
        if is_cluster:
            ax.text(2004.3, baseline + 0.02, f'{y_min}',
                    va='center', ha='right', fontsize=8, color='black')
            ax.text(2004.3, baseline + current_box_height - 0.02, f'{y_max}',
                    va='center', ha='right', fontsize=8, color='black')
        else:
            ax.text(2004.3, baseline + 0.02, f'{y_min}',
                    va='center', ha='right', fontsize=8, color='black')
            ax.text(2004.3, baseline + 0.38, f'{y_max}',
                    va='center', ha='right', fontsize=8, color='black')

    # X 轴设置
    ax.set_xlim(2005, 2023)
    ax.set_xticks([2005, 2014, 2023])
    ax.set_xticklabels([2005, 2014, 2023], fontsize=11, rotation=60, ha='center')
    ax.set_xlabel('Year', fontsize=13, labelpad=3)

    # Y 轴设置
    ax.set_ylim(0, total_plot_height)
    ax.set_yticks([])
    ax.set_ylabel('Countries', fontsize=13, labelpad=55)

    # 边框设置（只保留底部）
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(BORDER_WIDTH)
    ax.tick_params(axis='x', colors='black', width=BORDER_WIDTH, length=4, pad=2)

    plt.tight_layout()

    # 保存图片
    policy_short = policy_short_mapping[policy_name]
    out_path = output_dir / f"{policy_short}_ridge_plot.png"
    fig.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.15)
    plt.close()
    print(f"  ✓ {out_path.name} 已保存 ({len(valid_countries)} 个国家 + 1 个 Cluster 空白框)")


# 主程序
print(f"\n开始生成山脊图...")
print(f"选定国家 (按自下而上层级):")
print(f"  - Cluster (空白框, Y 高度 = 0.4 + 0.3)")
print(f"  - DEU (德国, 欧洲)")
print(f"  - JPN (日本, 亚洲)")
print(f"  - MEX (墨西哥, 北美洲)")
print(f"  - NZL (新西兰, 大洋洲)")
print(f"  - CHL (智利, 南美洲)")
print(f"  - ARG (阿根廷, 南美洲)\n")

for policy_idx, (policy_name, policy_config) in enumerate(policies.items(), 1):
    print(f"[{policy_idx}/{len(policies)}] {policy_name}")
    plot_ridge_chart(policy_name, policy_config, selected_countries, df)

print(f"\n✅ 完成！共生成 {len(policies)} 个山脊图")
print(f"📁 保存位置: {output_dir}\n")



开始生成山脊图...
选定国家 (按自下而上层级):
  - Cluster (空白框, Y 高度 = 0.4 + 0.3)
  - DEU (德国, 欧洲)
  - JPN (日本, 亚洲)
  - MEX (墨西哥, 北美洲)
  - NZL (新西兰, 大洋洲)
  - CHL (智利, 南美洲)
  - ARG (阿根廷, 南美洲)

[1/2] Transport – market-based instruments
  ✓ Transport_MBI_ridge_plot.png 已保存 (6 个国家 + 1 个 Cluster 空白框)
[2/2] Public Research, Development and Demonstration


C:\Users\28274\AppData\Local\Temp\ipykernel_17968\2067140164.py:78: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')
C:\Users\28274\AppData\Local\Temp\ipykernel_17968\2067140164.py:78: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


  ✓ Public_RD_ridge_plot.png 已保存 (6 个国家 + 1 个 Cluster 空白框)

✅ 完成！共生成 2 个山脊图
📁 保存位置: f:\Desktop\CAMPF_Supplementary\data\2-2-Country_Policy_Trends



In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pathlib import Path
import os
from general_pic_setup import setup_mpl_single2
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.collections import LineCollection

# ===================== 基础样式设置 =====================
setup_mpl_single2()
mpl.rcParams['axes.titlesize'] = 'medium'

# ===================== 配色 & 工具函数 =====================
# Nature 风格配色（用于 Cluster 小图）
nature_colors = [
    '#E64B35', "#6917C2", '#00A087', '#3C5488',
    '#F39B7F', '#8491B4', '#91D1C2', '#DC0000',
    '#7E6148', '#B09C85', '#E18727', '#20854E',
    '#0072B5', '#BC3C29', '#6F99AD',
]

def lighten_color(color, amount=0.7):
    """将颜色变浅, 用于圆圈填充"""
    try:
        c = mcolors.to_rgb(color)
        c = tuple([c[i] + (1 - c[i]) * amount for i in range(3)])
        return c
    except Exception:
        return color


def load_cluster_data(data_dir: Path):
    """加载聚类结果 & 详细时间序列数据"""
    cluster_assignment_path = data_dir / "3-1-L2_Policy_Clustering_pic" / "3-1_cluster_assignment_table.csv"
    cluster_df = pd.read_csv(cluster_assignment_path, index_col=0, encoding='utf-8-sig')

    cluster_detail_path = data_dir / "3-1-L2_Policy_Clustering_countries.csv"
    detail_df = pd.read_csv(cluster_detail_path, encoding='utf-8-sig')

    return cluster_df, detail_df


def draw_cluster_in_box(ax, baseline, height, policy_name, y_min, y_max,
                        cluster_df, detail_df):
    """
    在给定的矩形框 (baseline, baseline+height) 内，绘制缩小版的
    "所有簇合并图"：每个 cluster 的平均线 + 该 cluster 内国家线 + overall average。
    只使用主图的 X 轴（Year） & Y 轴范围 (y_min, y_max) 做线性缩放。
    """
    # 若没有对应政策的聚类结果，直接返回（只保留空框）
    if cluster_df is None or detail_df is None:
        return
    if policy_name not in cluster_df.index:
        return

    # 1. 聚类分配（这一行的列名是国家，值是簇 ID）
    cluster_assignments = cluster_df.loc[policy_name].dropna()
    if cluster_assignments.empty:
        return

    # 2. 取该政策对应的详细时间序列
    l2_data = detail_df[detail_df['L2政策中文名'] == policy_name].copy()
    if l2_data.empty:
        return

    # 3. 构建每个国家的时间序列： {country: Series(year -> value)}
    country_matrices = {}
    for country in cluster_assignments.index:
        country_data = l2_data[l2_data['国家'] == country].sort_values('年份')
        if country_data.empty:
            continue
        series = pd.Series(country_data['占比'].values,
                           index=country_data['年份'].values)
        country_matrices[country] = series

    if not country_matrices:
        return

    # 4. 年份列表（按升序）
    years = sorted(l2_data['年份'].unique().tolist())

    # 5. 按簇 ID 分组国家
    cluster_dict = {}
    for country, cid in cluster_assignments.items():
        if country in country_matrices:
            cid = int(cid)
            cluster_dict.setdefault(cid, []).append(country)

    if not cluster_dict:
        return

    # 6. 计算整体平均（overall average）
    overall_sum = pd.Series(0.0, index=years, dtype=float)
    overall_cnt = pd.Series(0, index=years, dtype=float)

    for country, series in country_matrices.items():
        for y in years:
            if y in series.index and not pd.isna(series[y]):
                overall_sum[y] += series[y]
                overall_cnt[y] += 1

    overall_avg = overall_sum / overall_cnt.replace(0, np.nan)

    # ========== 开始在 box 内绘图 (缩小版) ==========
    # 映射函数：真实值 value -> box 内 y 坐标
    def map_to_box_y(values):
        values = np.asarray(values, dtype=float)
        # 对超出范围的值进行截断
        values_clipped = np.clip(values, y_min, y_max)
        norm = (values_clipped - y_min) / (y_max - y_min)  # 0~1
        return baseline + norm * height

    # 用于存储图例
    legend_handles = []

    # 按 cluster ID 排序，分层绘制
    for idx, cid in enumerate(sorted(cluster_dict.keys())):
        countries = cluster_dict[cid]

        color = nature_colors[idx % len(nature_colors)]
        fill_color = lighten_color(color, amount=0.7)

        zorder_base = 200 + idx * 10

        # 6.1 先画各国曲线（更浅的颜色）
        for country in countries:
            series = country_matrices.get(country)
            if series is None:
                continue

            vals = np.array([series.get(y, np.nan) for y in years], dtype=float)
            mask = ~np.isnan(vals)
            if mask.sum() < 2:
                continue

            x_valid = np.array(years)[mask]
            y_valid = vals[mask]
            y_mapped = map_to_box_y(y_valid)

            ax.plot(x_valid, y_mapped,
                    color=color,
                    alpha=0.12,  # 从 0.25 降低到 0.12，使背景线更淡
                    linewidth=0.8,  # 从 1.0 降低到 0.8
                    zorder=zorder_base + 1)

        # 6.2 再画该 cluster 的平均线（带圆点，圆点改小到 2）
        cluster_sum = pd.Series(0.0, index=years, dtype=float)
        cluster_cnt = pd.Series(0, index=years, dtype=float)
        for country in countries:
            series = country_matrices.get(country)
            if series is None:
                continue
            for y in years:
                if y in series.index and not pd.isna(series[y]):
                    cluster_sum[y] += series[y]
                    cluster_cnt[y] += 1
        cluster_avg = cluster_sum / cluster_cnt.replace(0, np.nan)

        vals_avg = cluster_avg.values.astype(float)
        mask_avg = ~np.isnan(vals_avg)
        if mask_avg.sum() >= 2:
            x_avg = np.array(years)[mask_avg]
            y_avg = vals_avg[mask_avg]
            y_avg_mapped = map_to_box_y(y_avg)

            line, = ax.plot(x_avg, y_avg_mapped,
                    marker='o',
                    color=color,
                    linewidth=1.2,  # 从 2.0 降低到 1.2，使线更细
                    markersize=4,   # 从 6 改为 2，使圆球更小
                    markerfacecolor=fill_color,
                    markeredgecolor=color,
                    markeredgewidth=0.8,  # 相应调小边缘宽度
                    zorder=zorder_base + 5)
            
            # 添加到图例列表
            legend_handles.append((line, f'Cluster {cid}'))

    # 6.3 画整体平均线（黑色虚线, 最高层，线宽变细）
    vals_overall = overall_avg.values.astype(float)
    mask_o = ~np.isnan(vals_overall)
    if mask_o.sum() >= 2:
        x_o = np.array(years)[mask_o]
        y_o = vals_overall[mask_o]
        y_o_mapped = map_to_box_y(y_o)

        overall_line, = ax.plot(x_o, y_o_mapped,
                color='#000000',
                linestyle='--',
                linewidth=1.2,  # 从 2.0 降低到 1.2
                alpha=0.9,
                dashes=(3, 2),
                zorder=500)
        
        # 将 overall average 也加入图例
        legend_handles.append((overall_line, 'Overall Avg'))

    # 6.4 绘制图例（放在 Cluster 框的右上角）
    if legend_handles:
        # 计算图例位置：使用数据坐标系
        legend_x = 2022  # X 位置（靠近右侧）
        legend_y = baseline + height * 0.95  # Y 位置（靠近顶部）
        
        # 创建图例，使用更小的字体和紧凑的布局
        legend = ax.legend(
            [h for h, _ in legend_handles],
            [l for _, l in legend_handles],
            loc='upper right',
            fontsize=6.5,
            frameon=False,
            borderpad=0.2,
            handlelength=1.3,
            handletextpad=0.5,
            bbox_to_anchor=(1.0, 1.0),
            bbox_transform=ax.transData
        )
        
        # 调整图例位置到 Cluster 框内
        legend.set_bbox_to_anchor((legend_x, legend_y), transform=ax.transData)


# ===================== 主 Ridge 图代码 =====================

# 路径配置
script_dir = Path(os.getcwd())
data_dir = script_dir.parent / "data"
output_dir = data_dir / "2-2-Country_Policy_Trends"
output_dir.mkdir(parents=True, exist_ok=True)

# 数据加载与预处理（国家政策占比时间序列，用于 Ridge）
df = pd.read_parquet(data_dir / "2-1-country_proportions.parquet")
country_col = 'REF_AREA'
year_col = 'TIME_PERIOD'

df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
df = df.dropna(subset=[year_col])
df = df[(df[year_col] >= 2005) & (df[year_col] <= 2023)].sort_values(year_col)

# 加载聚类数据（用于 Cluster 框的小图）
cluster_df, cluster_detail_df = load_cluster_data(data_dir)

# 选定的 6 个国家（注意：ARG 和 CHL 已互换顺序）
# 自上而下视觉顺序：DEU, JPN, MEX, NZL, ARG, CHL, 最下面是 Cluster
selected_countries = [
    "DEU",  # 德国 - 欧洲
    "JPN",  # 日本 - 亚洲
    "MEX",  # 墨西哥 - 北美洲
    "NZL",  # 新西兰 - 大洋洲
    "ARG",  # 阿根廷 - 南美洲 (已提前)
    "CHL",  # 智利 - 南美洲 (在 ARG 下面)
]

# 政策配置（含 y 轴范围）
policies = {
    "Transport – market-based instruments": {
        "label": "Transport Policy",
        "y_range": (0, 0.2)
    },
    "Public Research, Development and Demonstration": {
        "label": "Research Development",
        "y_range": (0, 0.6)
    }
}

policy_short_mapping = {
    "Transport – market-based instruments": "Transport_MBI",
    "Public Research, Development and Demonstration": "Public_RD"
}

# 绘图参数
BORDER_WIDTH = 0.6
CURVE_WIDTH = 1.5   # 单个国家曲线线宽
box_height = 0.4    # 普通国家框高度
gap_between = 0.01  # 框之间的间隔
empty_box_extra_height = 0.3  # Cluster 额外高度（+0.3）


def plot_ridge_chart(policy_name, policy_config, selected_countries, df,
                     cluster_df, cluster_detail_df):
    fig, ax = plt.subplots(figsize=(4, 9))
    cmap = cm.get_cmap('coolwarm')

    y_min, y_max = policy_config["y_range"]

    # 先筛掉没有数据的国家
    valid_countries = [
        c for c in selected_countries
        if c in df[country_col].values
        and not df[df[country_col] == c][policy_name].isna().all()
    ]

    # 所有层：最下面是 Cluster，其上是各个国家
    layers = ["CLUSTER"] + valid_countries

    # 为每一层设置高度：Cluster 的框高 = 0.4 + 0.3，其余 = 0.4
    heights = []
    for code in layers:
        if code == "CLUSTER":
            heights.append(box_height + empty_box_extra_height)
        else:
            heights.append(box_height)

    # 计算每一层的 baseline，使得相邻框之间的间隔都 = gap_between
    baselines = [0.0] * len(layers)
    for i in range(1, len(layers)):
        baselines[i] = baselines[i-1] + heights[i-1] + gap_between

    # 总高度
    total_plot_height = baselines[-1] + heights[-1] if layers else 1.0

    for idx, code in enumerate(layers):
        baseline = baselines[idx]
        current_box_height = heights[idx]
        is_cluster = (code == "CLUSTER")

        # 1. 绘制 L 型边框
        ax.plot([2005, 2005], [baseline, baseline + current_box_height],
                color='black', linewidth=BORDER_WIDTH, zorder=idx - 1)
        ax.plot([2005, 2023], [baseline, baseline],
                color='black', linewidth=BORDER_WIDTH, zorder=idx - 1)

        # 2. 非 Cluster：绘制国家曲线
        if not is_cluster:
            df_country = df[df[country_col] == code]
            years = df_country[year_col].values
            values = df_country[policy_name].values

            mask = ~np.isnan(values)
            years, values = years[mask], values[mask]

            if len(values) > 1:
                values_clipped = np.clip(values, y_min, y_max)
                values_normalized = (values_clipped - y_min) / (y_max - y_min) * box_height
                points = np.array([years, baseline + values_normalized]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                n_segments = len(segments)
                if n_segments > 1:
                    colors_gradient = [cmap(i / (n_segments - 1)) for i in range(n_segments)]
                else:
                    colors_gradient = [cmap(0.5)]
                lc = LineCollection(segments, colors=colors_gradient,
                                    linewidth=CURVE_WIDTH, alpha=0.95, zorder=idx + 100)
                ax.add_collection(lc)
        else:
            # 3. Cluster 框：在框内绘制缩小版"所有簇合并图"（含图例）
            draw_cluster_in_box(
                ax=ax,
                baseline=baseline,
                height=current_box_height,
                policy_name=policy_name,
                y_min=y_min,
                y_max=y_max,
                cluster_df=cluster_df,
                detail_df=cluster_detail_df
            )

        # 4. 左侧标签
        label_y_offset = 0.2  # 保持"正常"高度，不因 Cluster 变动
        norm_y = (baseline + label_y_offset) / total_plot_height
        display_label = "Cluster" if is_cluster else code
        ax.text(-0.12, norm_y, display_label,
                transform=ax.transAxes, va='center', ha='right',
                fontsize=11, color='black', weight='bold')

        # 5. Y 轴刻度文字
        if is_cluster:
            ax.text(2004.3, baseline + 0.02, f'{y_min}',
                    va='center', ha='right', fontsize=8, color='black')
            ax.text(2004.3, baseline + current_box_height - 0.02, f'{y_max}',
                    va='center', ha='right', fontsize=8, color='black')
        else:
            ax.text(2004.3, baseline + 0.02, f'{y_min}',
                    va='center', ha='right', fontsize=8, color='black')
            ax.text(2004.3, baseline + 0.38, f'{y_max}',
                    va='center', ha='right', fontsize=8, color='black')

    # X 轴设置
    ax.set_xlim(2005, 2023)
    ax.set_xticks([2005, 2014, 2023])
    ax.set_xticklabels([2005, 2014, 2023], fontsize=11, rotation=60, ha='center')
    ax.set_xlabel('Year', fontsize=13, labelpad=3)

    # Y 轴设置
    ax.set_ylim(0, total_plot_height)
    ax.set_yticks([])
    ax.set_ylabel('Countries', fontsize=13, labelpad=55)

    # 边框设置（只保留底部）
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(BORDER_WIDTH)
    ax.tick_params(axis='x', colors='black', width=BORDER_WIDTH, length=4, pad=2)

    plt.tight_layout()

    # 保存图片
    policy_short = policy_short_mapping[policy_name]
    out_path = output_dir / f"{policy_short}_ridge_plot.png"
    fig.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.15)
    plt.close()
    print(f"  ✓ {out_path.name} 已保存 ({len(valid_countries)} 个国家 + 1 个 Cluster 小图)")


# ===================== 主程序 =====================

print(f"\n开始生成山脊图...")
print("国家顺序（自上而下）:")
print("  DEU, JPN, MEX, NZL, ARG, CHL, 最下面是 Cluster（聚类小图）\n")

for policy_idx, (policy_name, policy_config) in enumerate(policies.items(), 1):
    print(f"[{policy_idx}/{len(policies)}] {policy_name}")
    plot_ridge_chart(
        policy_name=policy_name,
        policy_config=policy_config,
        selected_countries=selected_countries,
        df=df,
        cluster_df=cluster_df,
        cluster_detail_df=cluster_detail_df
    )

print(f"\n✅ 完成！共生成 {len(policies)} 个山脊图")
print(f"📁 保存位置: {output_dir}\n")



开始生成山脊图...
国家顺序（自上而下）:
  DEU, JPN, MEX, NZL, ARG, CHL, 最下面是 Cluster（聚类小图）

[1/2] Transport – market-based instruments


C:\Users\28274\AppData\Local\Temp\ipykernel_17968\43542285.py:284: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


  ✓ Transport_MBI_ridge_plot.png 已保存 (6 个国家 + 1 个 Cluster 小图)
[2/2] Public Research, Development and Demonstration


C:\Users\28274\AppData\Local\Temp\ipykernel_17968\43542285.py:284: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


  ✓ Public_RD_ridge_plot.png 已保存 (6 个国家 + 1 个 Cluster 小图)

✅ 完成！共生成 2 个山脊图
📁 保存位置: f:\Desktop\CAMPF_Supplementary\data\2-2-Country_Policy_Trends

